<a href="https://colab.research.google.com/github/lealkedna/DesafioLLMs/blob/main/Desafio_tecnico_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objetivo

Desenvolver uma solução que acesse pelo menos três plataformas de Modelos
de Linguagem de Grande Escala (LLMs) diferentes, gere respostas para uma mesma
pergunta e realize uma análise comparativa da qualidade das respostas

In [1]:
 !pip install aisuite[all]
 !pip install tabulate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


Configurações API Keys como as chaves de API como variavéis de ambiente


In [2]:
import os
from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("GROQ_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = getpass("ANTHROPIC_API_KEY")

GROQ_API_KEY··········
ANTHROPIC_API_KEY··········


Criação de uma função ask para interagir com o modelo de linguagem

In [3]:
import aisuite as ai

client = ai.Client()

messages = [
    {"role": "system", "content": "responda de forma clara e objetiva."},
    {"role": "user", "content": "Hi"},
]

response = client.chat.completions.create(model="groq:deepseek-r1-distill-qwen-32b", messages=messages)

 # model="gpt-3.5-turbo",
print(response.choices[0].message.content)

def ask(message, sys_message="You are a helpful agent.",
        model="groq:deepseek-r1-distill-qwen-32b"):
    messages = [
        {"role": "system", "content": sys_message},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(model=model, messages=messages)

    return response.choices[0].message.content

Claro! Em que posso ajudá-lo?


## Coleta respostas dos modelos

In [6]:
from pprint import pprint
import json


providers = [
    'groq:llama-3.3-70b-specdec',
    'anthropic:claude-3-5-haiku-20241022',
    'groq:deepseek-r1-distill-qwen-32b',
    'groq:mixtral-8x7b-32768',
    # 'groq:qwen-2.5-coder-32b'
]

perguntas = [
    "O que vc sabe sobre a cognitiva Brasil?",
    "Qual o valor da cotação do dólar hoje?",
    "Quais são as principais notícias do mundo hoje?"
    #pode adicionar mais perguntas aqui
]

# Armazenar respostas de cada modelo
respostas_por_pergunta = {pergunta: {provider: None for provider in providers} for pergunta in perguntas}

# Função para coletar respostas dos modelos
def coletar_respostas():
    for provider in providers:
        for pergunta in perguntas:
            try:
                resposta = ask(pergunta, model=provider)
                respostas_por_pergunta[pergunta][provider] = resposta
            except Exception as e:
                respostas_por_pergunta[pergunta][provider] = f"Erro ao acessar {provider}: {str(e)}"
    print("Respostas coletadas com sucesso!")

coletar_respostas()

# Exibir em formato JSON organizado
for pergunta in perguntas:
    print(f"\n📌 Pergunta: {pergunta}")
    print(json.dumps(respostas_por_pergunta[pergunta], indent=4, ensure_ascii=False))

Respostas coletadas com sucesso!

📌 Pergunta: O que vc sabe sobre a cognitiva Brasil?
{
    "groq:llama-3.3-70b-specdec": "Olá! A Cognitiva Brasil é uma empresa brasileira de soluções de inteligência artificial (IA) e ciência de dados. Aqui está o que sei sobre a empresa:\n\n**História e Fundação**: A Cognitiva Brasil foi fundada em 2015, em São Paulo, Brasil. A empresa nasceu com o objetivo de ajudar as organizações a extrair valor dos seus dados e melhorar a tomada de decisões por meio da aplicação de técnicas de inteligência artificial e ciência de dados.\n\n**Atividades e Serviços**: A Cognitiva Brasil oferece uma variedade de serviços e soluções em inteligência artificial e ciência de dados, incluindo:\n\n* Desenvolvimento de modelos de previsão e análise de dados\n* Implementação de soluções de aprendizado de máquina (Machine Learning)\n* Análise de dados e visualização de informações\n* Desenvolvimento de sistemas de recomendação\n* Otimização de processos e automatização de tar

## Ranquear Respostas e Exibir Resultados



In [7]:
import re
from tabulate import tabulate

def extrair_ranking(resposta, num_respostas):
    match = re.search(r'\[(\d+,\s*)*\d+\]', resposta)
    if match:
        try:
            ranking = eval(match.group())
            if isinstance(ranking, list) and len(ranking) == num_respostas:
                if all(isinstance(x, int) and 1 <= x <= num_respostas for x in ranking):
                    return ranking
        except:
            pass
    return None

def ranquear_respostas(pergunta, respostas, model):

    respostas_formatadas = "\n".join([f"{idx + 1}. {resposta}" for idx, resposta in enumerate(respostas)])

    prompt = f"""
    Abaixo estão as respostas de diferentes modelos para a pergunta: '{pergunta}'.
    Por favor, ranqueie as respostas de 1 a {len(respostas)} (sendo 1 a melhor) com base nos seguintes critérios:
    - Clareza e coerência
    - Precisão da informação
    - Criatividade ou profundidade
    - Consistência gramatical

    Respostas:
    {respostas_formatadas}

    Por favor, retorne apenas uma lista de números na ordem do ranking (exemplo: [2, 1, 3]).
    """

    try:
        resposta_modelo = ask(prompt, model=model)
        ranking = extrair_ranking(resposta_modelo, len(respostas))
        return ranking
    except Exception as e:
        print(f"Erro ao ranquear respostas com {model}: {str(e)}")
        return None

# Armazenar rankings de cada modelo
rankings_por_pergunta = {pergunta: {provider: None for provider in providers} for pergunta in perguntas}


for pergunta in perguntas:
    respostas = [respostas_por_pergunta[pergunta][provider] for provider in providers]

    for provider in providers:
        ranking = ranquear_respostas(pergunta, respostas, provider)
        rankings_por_pergunta[pergunta][provider] = ranking


def calcular_ranking_final(rankings):
    pontuacoes = {idx: 0 for idx in range(len(providers))}
    rankings_validos = [ranking for ranking in rankings.values() if ranking is not None]

    if not rankings_validos:
        return list(range(len(providers))), pontuacoes

    for ranking in rankings_validos:
        for posicao, idx in enumerate(ranking):
            pontuacoes[idx - 1] += posicao + 1


    ranking_final = sorted(pontuacoes.keys(), key=lambda x: pontuacoes[x])
    return ranking_final, pontuacoes

from tabulate import tabulate


print("\n🏆 Rankings Finais:")
for pergunta in perguntas:
    print(f"\n📌 Pergunta: {pergunta}\n")

    # Calcular o ranking final
    ranking_final, pontuacoes = calcular_ranking_final(rankings_por_pergunta[pergunta])

    # Preparar dados para a tabela
    tabela_ranking = []
    for posicao, idx in enumerate(ranking_final, start=1):
        provider = providers[idx]
        tabela_ranking.append([posicao, provider,])


    print(tabulate(tabela_ranking, headers=["🏅 Posição", "🤖 Modelo"], tablefmt="fancy_grid"))



🏆 Rankings Finais:

📌 Pergunta: O que vc sabe sobre a cognitiva Brasil?

╒══════════════╤═════════════════════════════════════╕
│   🏅 Posição │ 🤖 Modelo                           │
╞══════════════╪═════════════════════════════════════╡
│            1 │ groq:llama-3.3-70b-specdec          │
├──────────────┼─────────────────────────────────────┤
│            2 │ anthropic:claude-3-5-haiku-20241022 │
├──────────────┼─────────────────────────────────────┤
│            3 │ groq:mixtral-8x7b-32768             │
├──────────────┼─────────────────────────────────────┤
│            4 │ groq:deepseek-r1-distill-qwen-32b   │
╘══════════════╧═════════════════════════════════════╛

📌 Pergunta: Qual o valor da cotação do dólar hoje?

╒══════════════╤═════════════════════════════════════╕
│   🏅 Posição │ 🤖 Modelo                           │
╞══════════════╪═════════════════════════════════════╡
│            1 │ groq:llama-3.3-70b-specdec          │
├──────────────┼────────────────────────────────────